<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Fallstudie_Juicy_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [101]:
! pip install -q pyscipopt

In [102]:

import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [103]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [104]:
! ls

sample_data  WS25_Supply_Chain_Optimierung


In [105]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [106]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [107]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [108]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [109]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [110]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [111]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [112]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [113]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


# Modell für JUICY AG

In [114]:

# Erstellen einer Modellinstanz
model = Model()

# Indexmengen

In [115]:
I = list(kapazitaeten.Produktionsstandort.unique())
J = list(nachfrage.Markt.unique())
A = list(kapazitaeten.Ausbaustufe.unique())

# Entscheidungsvariablen

In [116]:
x = {}
for i in I:
  for j in J:
    x[i,j] = model.addVar(vtype='I', name = f"Produktionsmenge am Standort {i} für Markt {j}")

y = {}
for i in I:
  for a in A:
    y[i,a] = model.addVar(vtype='B', name = f"Wenn Werk {i} eine Ausbaustufe {a} baut")

len(model.getVars())


35

# Parameter

In [117]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]
print("Produktionskosten:", cp)

#Der Code verwandelt eine Tabelle in einen zweidimensionalen Parameter der direkt im Optimierungsmodell verwendet werden kann. MultiIndex → Tuple-Schlüssel → Modellparamete
#Warum nicht wie im Beispiel davor? Du kannst das so machen – aber nur, wenn dein Parameter eindimensional ist. Hier brauchst du zweidimensionale Parameter.

# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]
print("Investitionen:", cf)


# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
print("Kapazitäten:", cap)


# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
print("Nachfragemenge:", d)

Produktionskosten: {('Charleston', 'USA'): 81, ('Charleston', 'Südamerika'): 92, ('Charleston', 'Europa'): 101, ('Charleston', 'Asien'): 130, ('Charleston', 'Afrika'): 115, ('Curitiba', 'USA'): 117, ('Curitiba', 'Südamerika'): 77, ('Curitiba', 'Europa'): 108, ('Curitiba', 'Asien'): 98, ('Curitiba', 'Afrika'): 100, ('Hamburg', 'USA'): 102, ('Hamburg', 'Südamerika'): 105, ('Hamburg', 'Europa'): 95, ('Hamburg', 'Asien'): 119, ('Hamburg', 'Afrika'): 111, ('Quanzhou', 'USA'): 115, ('Quanzhou', 'Südamerika'): 125, ('Quanzhou', 'Europa'): 90, ('Quanzhou', 'Asien'): 59, ('Quanzhou', 'Afrika'): 74, ('East London', 'USA'): 142, ('East London', 'Südamerika'): 100, ('East London', 'Europa'): 103, ('East London', 'Asien'): 105, ('East London', 'Afrika'): 71}
Investitionen: {('Charleston', 'klein'): 6000, ('Charleston', 'groß'): 9000, ('Curitiba', 'klein'): 4500, ('Curitiba', 'groß'): 6750, ('Hamburg', 'klein'): 6500, ('Hamburg', 'groß'): 9750, ('Quanzhou', 'klein'): 4100, ('Quanzhou', 'groß'): 6150